In [ ]:
import hashlib

In [ ]:
# Merkle Tree Node 클래스
class MerkleNode:
    def __init__(self, left=None, right=None, value=None):
        """
        Merkle Tree의 노드를 초기화합니다.
        Args:
            left (MerkleNode): 왼쪽 자식 노드.
            right (MerkleNode): 오른쪽 자식 노드.
            value (str): 노드의 해시 값.
        """
        self.left = left
        self.right = right
        self.value = value

In [ ]:
# Merkle Tree 클래스
class MerkleTree:
    def __init__(self, transactions):
        """
        Merkle Tree를 생성합니다.
        Args:
            transactions (list): 트랜잭션 데이터 리스트.
        """
        self.transactions = transactions
        self.root = self._build_tree(transactions)

    def _build_tree(self, transactions):
        """
        Merkle Tree를 재귀적으로 빌드합니다.
        Args:
            transactions (list): 트랜잭션 데이터 리스트.
        Returns:
            MerkleNode: Merkle Tree의 루트 노드.
        """
        if len(transactions) == 1:
            return MerkleNode(value=self._hash(transactions[0]))

        # 트랜잭션 리스트가 홀수면 마지막 항목을 복제
        if len(transactions) % 2 != 0:
            transactions.append(transactions[-1])

        # 자식 노드 생성
        left_nodes = transactions[:len(transactions) // 2]
        right_nodes = transactions[len(transactions) // 2:]

        left = self._build_tree(left_nodes)
        right = self._build_tree(right_nodes)

        # 부모 노드 생성
        parent_hash = self._hash(left.value + right.value)
        return MerkleNode(left=left, right=right, value=parent_hash)

    def _hash(self, data):
        """
        데이터를 해싱합니다.
        Args:
            data (str): 해싱할 데이터.
        Returns:
            str: 해시 값.
        """
        return hashlib.sha256(data.encode()).hexdigest()

    def get_root(self):
        """
        Merkle Root를 반환합니다.
        Returns:
            str: Merkle Root 값.
        """
        return self.root.value

    def validate_transaction(self, transaction):
        """
        특정 트랜잭션이 Merkle Tree에 포함되어 있는지 검증합니다.
        Args:
            transaction (str): 검증할 트랜잭션.
        Returns:
            bool: 트랜잭션이 Merkle Tree에 포함되어 있는지 여부.
        """
        current_hash = self._hash(transaction)
        current_node = self.root

        while current_node.left or current_node.right:
            if current_hash in (current_node.left.value, current_node.right.value):
                current_hash = self._hash(current_node.left.value + current_node.right.value)
                if current_hash == self.root.value:
                    return True
            else:
                return False
        return current_hash == self.root.value

In [ ]:
# Merkle Tree 실습 예제
if __name__ == "__main__":
    # 1. 트랜잭션 데이터 리스트
    transactions = ["tx1", "tx2", "tx3", "tx4"]

    # 2. Merkle Tree 생성
    merkle_tree = MerkleTree(transactions)
    print("Merkle Root:", merkle_tree.get_root())

    # 3. 트랜잭션 검증
    valid_transaction = "tx1"
    invalid_transaction = "tx5"
    print(f"\nIs '{valid_transaction}' valid?", merkle_tree.validate_transaction(valid_transaction))
    print(f"Is '{invalid_transaction}' valid?", merkle_tree.validate_transaction(invalid_transaction))